<a id="top"></a>
# Arrhythmia Prediction

Electrocardiograms (ECG) are records of the electrical activity of the heart gathered from electrodes placed on the skin. They are commonly used for medical monitoring and diagnosis.

This ECG demo is based on the model developed by the [Stanford ML group](https://stanfordmlgroup.github.io/projects/ecg2/) using the [PhysioNet 2017 challenge dataset](https://www.physionet.org/content/challenge-2017/1.0.0/).  The [GitHub repository](https://github.com/awni/ecg) contains the original code and resources for training models.

![Example ECG](figures/A00150.gif) 

Awni Y Hannun, Pranav Rajpurkar, Masoumeh Haghpanahi, Geoffrey H Ti-son, Codie Bourn, Mintu P Turakhia, and Andrew Y Ng. Cardiologist-level arrhythmia  detection  and  classification  in  ambulatory electrocardiograms using a deep neural network. Nature Medicine, 25(1):65, 2019. https://www.nature.com/articles/s41591-018-0268-3


## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "Arrhythmia_Prediction.ipynb", "openvino-dev-latest", ['openvino-lts'])

## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    - **Arrhythmia_prediction.ipynb** - This Jupyter* Notebook
    - **figures/A00150.gif** - Example ECG shown at beginning of notebook
    - **figures/EKG_info.svg** - Image showing the different parts of ECG rhythms
    - **python/inference.py** - Python* code for inference jobs sent to edge nodes
    - **python/keras_inference.py** - Python* code for inference done using Keras with a Tensorflow backend
    - **python/load.py** - Python* code for loading files from the dataset
    - **python/openvino_inference.py** - Python* code inference using OpenVINO
    - **python/tensorflow_conversion.py** - Python* code for converting the Keras model into a tensorflow protobuf format
    - **data/reference.csv** - Text file with class labels for the dataset
    - **requirements.txt** - Python* requirements file
    - **/data/ecg/0.427-0.863-020-0.290-0.899.hdf5** - Pre-trained model file
    - **/data/ecg/training/*** - Location of the dataset files


It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter* Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>


## Requirements and Imports
Running the inference requires Keras and serveral other packages to be installed. Run the following cells to ensure that all dependencies are satisfied.

In [ ]:
print("Installing requirements")
!python3 -m pip --no-cache-dir install -r requirements.txt --user

In [ ]:
import json
import os
import sys
import subprocess
from qarpo.demoutils import *

import matplotlib.pyplot as plt
import matplotlib.animation as ani
import numpy as np
import scipy.stats as sst
import scipy.io as sio
from IPython.display import HTML
from matplotlib.ticker import MultipleLocator, AutoMinorLocator

os.makedirs("results", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("models", exist_ok=True)

## Dataset Description

This network is used to detect arrhythmias from ECG time series data. The model was trained using the PhysioNet Computing in Cardiology Challenge 2017 (CINC17) dataset, which has four distinct classes which are as follows:

N - normal sinus rhythm

A - atrial fibrillation (AF)

O - an alternative rhythm

~ - too noisy to be classified

Although this dataset only has four distinct classifications for the ECG records, it is possible to train the model to distinguish between different types of arrhythmias if labels are provided to distinguish between different types.

Both the Keras and the OpenVINO models will be run using a subset of the original test data that excludes any examples below a specified length. Although the Keras model can take inputs of variable size, we use the same input set as the OpenVINO examples for consistency. 

## Visualizing the Information

In this section we will see examples of the four different classes of arrhythmias and some of their distinguishing characteristics.

The code below will convert the time series data into short animations which illustrate what each class looks like. Note that processing can take some time (~1-2 mins).

In [ ]:
def create_ecg_animation(filename, y):
    step_size = 12
    x_lim = 1800
    scale_factor = 7
    num_frames = x_lim // step_size
    x =  range(0,(x_lim*scale_factor),scale_factor)
    y_max = 1.11*max(np.amax(y), abs(np.amin(y)))

    plt.ioff()
    
    #set up the figure
    fig, ax = plt.subplots()
    line, = ax.plot([], [], color='k')
    
    ax.tick_params(axis="both", which="both", length=0.0, labelbottom=False, labelleft=False)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Create the grid with 4x4 grid squares
    aspect_ratio = 4200 / (2*y_max)
    ax.xaxis.set_major_locator(MultipleLocator(400))
    ax.yaxis.set_major_locator(MultipleLocator(400 / aspect_ratio))
    ax.xaxis.set_minor_locator(AutoMinorLocator(4))
    ax.yaxis.set_minor_locator(AutoMinorLocator(4))
    ax.grid(which='major', linestyle='-', axis='both')
    ax.grid(which='minor', linewidth='0.5', axis='both', color='lightgray')

    # Settings for figure size
    ax.set_xlim(1, x_lim*scale_factor)
    ax.set_ylim(-y_max, y_max)
    fig.set_figheight(4.5)
    fig.set_figwidth(13.5)
    fig.tight_layout()
    
    canvas_width, canvas_height = fig.canvas.get_width_height()

    def update(num):
        offset = int(num // (x_lim / step_size)) + 2
        index = int(num % (x_lim / step_size))
        line.set_data(x[:(step_size*index)], y[(x_lim*offset):(x_lim*offset+step_size*index)])

    # Open an ffmpeg process
    cmdstring = ('ffmpeg', 
                 '-y', '-r', '25', # 25fps
                 '-s', '%dx%d' % (canvas_width, canvas_height), # size of image string
                 '-pix_fmt', 'argb', # format
                 '-f', 'rawvideo',  '-i', '-', # tell ffmpeg to expect raw video from the pipe
                 '-preset', 'ultrafast',
                 '-vcodec', 'h264', 'figures/' + filename) # output encoding
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE)

    # Draw frames and write to the pipe
    for frame in range(num_frames):
        # draw the frame
        update(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()

        # write to pipe
        p.stdin.write(string)

    # Finish up
    p.communicate()
    plt.close(fig)


### Normal Sinus Rhythm

Normal ECG rhythms consist of four distinct sections: P wave, QRS complex, T wave, and U wave.

<figure>
<img src="figures/EKG_info.svg" height=40%, width=40%/>
<figcaption style="text-align:center"><a href="https://commons.wikimedia.org/wiki/File:EKG_Complex_en.svg" title="via Wikimedia Commons">ECG Complex</a> [<a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA</a>]</figcaption>
</figure>

The P wave represents atrial depolarization.  
The QRS complex represents ventricular depolarization.  
The T wave represents ventricular repolarization.  
The U wave represents papillary muscle repolarization.  

In [ ]:
ecg_normal = sio.loadmat('/data/ecg/training/A00001.mat')['val'].squeeze()

create_ecg_animation('ecg_normal.mp4', ecg_normal)

HTML('''
    <video alt="test" controls autoplay loop>
        <source src="figures/ecg_normal.mp4" type="video/mp4">
    </video>
''')

### Atrial Fibrilation

Atrial fibrilation is usually distinguished by irregular intervals between heart beats, rapid heart rate, and lack of a P wave.

In [ ]:
ecg_af = sio.loadmat('/data/ecg/training/A00004.mat')['val'].squeeze()

create_ecg_animation("ecg_af.mp4", ecg_af)

HTML('''
    <video alt="test" controls autoplay loop>
        <source src="figures/ecg_af.mp4" type="video/mp4">
    </video>
''')

### Other Rhythm

For this dataset, all non-AF abnormal rhythms are classified as other rhythms. 

In [ ]:
ecg_other = sio.loadmat('/data/ecg/training/A00077.mat')['val'].squeeze()

create_ecg_animation("ecg_other.mp4", ecg_other)

HTML('''
    <video alt="test" controls autoplay loop>
        <source src="figures/ecg_other.mp4" type="video/mp4">
    </video>
''')

### Too noisy to be classified

This final classification includes data that has too much noise to have any distinguishable pattern.

In [ ]:
ecg_undef = sio.loadmat('/data/ecg/training/A01246.mat')['val'].squeeze()

create_ecg_animation("ecg_undef.mp4", ecg_undef)

HTML('''
    <video alt="test" controls autoplay loop>
        <source src="figures/ecg_undef.mp4" type="video/mp4">
    </video>
''')

## Running Prediction Using Keras

In this section we will run all of the sample data through Keras using a Tensorflow backend. After getting predictions from the model, we will compare it to the ground truth labels to measure accuracy. The inference it done by running the [keras_inference.py](./python/keras_inference.py) script.

In [ ]:
!python3 python/keras_inference.py

## Converting Keras Model to an OpenVINO Intermediate Representation

Next we want to run the model through the OpenVINO model optimizer to produce a Intermediate Representation (IR) that can be used to run inference using the OpenVINO inference engine. However, in order to make that possible we first have to convert the model to a format that is supported by the model optimizer. 

`hdf5` Keras → `pb` Tensorflow → `IR` OpenVINO

 Running the following cell takes the original keras model file `.hdf5` and converts it to a tensorflow frozen protobuf format `.pb` and will print out a summary of the original model for reference. The script for this conversion process can be found at [tensorflow_conversion.py](./python/tensorflow_conversion.py).

In [ ]:
!python3 python/tensorflow_conversion.py

Now that we have a `.pb` model file, we can pass that to the model optimizer for conversion. Running the model optimizer will generate a `.xml` and `.bin` which represent the network topology and model weights. 

We need to specify the size of the input for the model optimizer and we use `[1,8960,1]` to match our sample length. We also specify the data type as FP16 to support inference on GPUs.

In [ ]:
!mo_tf.py \
--input_model models/output_graph.pb                                   \
--output_dir models/                                                   \
--input_shape "[1,8960,1]"                                             \
--data_type FP16

## Running Inference Using OpenVINO

The OpenVINO model only takes input of a specific size so we truncate all of the data that is above that size before feeding it into the model. The script for running the OpenVINO inference can be found at [openvino_inference.py](./python/openvino_inference.py)

In [ ]:
!python3 python/openvino_inference.py

### Visualizing the Results

The following cell creates a visualization to show the results of the network's predictions. Different colors are used to represent the predicted class (N - blue, A - red, O - yellow, ~ - gray) with a score for the confidence in the prediction at the bottom of each section. The final predicted class is determined by taking the most commonly predicted class across all of the time slices for the sample. Only the first 20 time slices are shown below to allow for greater visual clarity.

In [ ]:
def visualize_result(sample, prediction, probabilities, actual):
    data_points = 256*20
    
    predicted = sst.mode(prediction)[0][0]
    y = sio.loadmat('/data/ecg/training/' + sample)['val'].squeeze()[:data_points]
    
    x =  range(0,data_points)
    y_max = 1.11*max(np.amax(y), abs(np.amin(y)))
    
    #set up the figure
    fig, ax = plt.subplots()
    line, = ax.plot(x, y, color='k')
    ax.title.set_text(sample + ', Predicted Class - ' + predicted + ', Actual Class - ' + actual)
    
    ax.tick_params(axis="both", which="both", length=0.0, labelbottom=False, labelleft=False)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Add axis lines
    ax.xaxis.set_major_locator(MultipleLocator(256))
    ax.grid(which='major', linestyle='-', axis='x', color='k')

    # Settings for figure size
    ax.set_xlim(1, data_points)
    ax.set_ylim(-y_max, y_max)
    fig.set_figheight(4)
    fig.set_figwidth(12)
    fig.tight_layout()
    
    i = 0
    for label, prob in zip(prediction[:20], probabilities[:20]):
        colors = {'A' : 'r', 'N' : 'b', 'O' : 'y', '~': 'k'}
        plt.text((i+.2)*256, -y_max, '%.3f' % prob)
        plt.axvspan(i*256, (i+1)*256, facecolor=colors[label], alpha=prob/3)
        i += 1
        
    plt.show(fig)    
    plt.close(fig)

def show_all_predictions():
    with open('results/predictions.json', 'r') as preds:
        preds = json.load(preds)
        items = list(preds.items())
        list(map(lambda x: visualize_result(x[0], x[1]['data'], x[1]['prob'], x[1]['actual']), items))

def show_prediction_subset(items):
    with open('results/predictions.json', 'r') as preds:
        preds = json.load(preds)
        list(map(lambda x: visualize_result(x, preds[x]['data'], preds[x]['prob'], preds[x]['actual']), items))

#### Predictions with accurate results

In [ ]:
show_prediction_subset(['A02261', 'A06779', 'A00593', 'A02479'])

#### Predictions with lower confidence or inaccurate results

In [ ]:
show_prediction_subset(['A00722', 'A04222', 'A08010', 'A08525'])

#### (Optional) View all of the evaluation data

Run the follwing cell to display all 300 of the samples and the predictions given by the model. Due to the number of samples, it may take a couple minutes to complete.

In [ ]:
show_all_predictions()

## Inference on the edge

All the code up to this point has been run on a development node based on an Intel Xeon Scalable processor. We will run the workload on other edge compute nodes represented in the IoT DevCloud by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is written in Bash, and will be executed directly on the edge compute node. For this example, we have written the job file for you in the notebook. It performs the classification using the script [inference.py](./python/inference.py).

In [ ]:
%%writefile prediction.sh

cd $PBS_O_WORKDIR

mkdir -p $1
OUTPUT_DIR=$1
DEVICE=$2

python3 python/inference.py -d ${DEVICE} -o ${OUTPUT_DIR}

### How jobs are submitted into the queue

Now that we have the job script, we can submit the jobs to edge compute nodes. In the IoT DevCloud, you can do this using the `qsub` command.
We can submit the job to 6 different types of edge compute nodes simultaneously or just one node at at time.

There are five options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets us name the job so that it is easier to distinguish between them.
- `-o` : this option lets us determine the path to be used for the standard output stream.
- `-e` : this option lets us determine the path to be used for the standard error stream.


The `-F` flag is used to pass in arguments to the job script.
The [prediction.sh](prediction.sh) script takes in 2 arguments:
1. the path to the directory for the output video and performance stats
2. targeted device (e.g. CPU, GPU, MYRIAD, HDDL)

The job scheduler will use the contents of `-F` flag as the argument to the job script.

If you are curious to see the available types of nodes on the IoT DevCloud, run the following cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

Here, the properties describe the node, and number on the left is the number of available nodes of that architecture.

### Job queue submission

The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

<br><div class=note><i><b>Note: </b>You can submit all the jobs at once or follow one at a time.</i></div> 

After submission, they will go into a queue and run as soon as the requested compute resources become available. 

<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>


#### Submitting to an edge compute node with an Intel Core CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel 
    Core i5-6500TE</a>. The inference workload will run on the CPU.

In [ ]:
job_id_core = !qsub prediction.sh -l nodes=1:idc001skl -F "results/core/ CPU" -N arrhythmia_core -e results/core/ -o results/core/   
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('./logs', job_id_core[0] + '_load.txt', "Data Loading", 0, 100)
    progressIndicator('./logs', job_id_core[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with an 8th Generation Intel Core CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/8th-gen-core-dev-kit">UP Xtreme Edge Compute Enabling Kit
    </a> edge node with a low power <a 
    href="https://ark.intel.com/content/www/us/en/ark/products/193554/intel-core-i7-8665ue-processor-8m-cache-up-to-4-40-ghz.html">Intel 
    Core i7-8865UE</a>. The inference workload will run on the CPU.


In [ ]:
job_id_core2 = !qsub prediction.sh -l nodes=1:idc014upxa10fx1 -F "results/core2/ CPU" -N arrhythmia_core2 -e results/core2/ -o results/core2/
print(job_id_core2[0]) 
#Progress indicators
if job_id_core2:
    progressIndicator('./logs', job_id_core2[0] + '_load.txt', "Data Loading", 0, 100)
    progressIndicator('./logs', job_id_core2[0]+'.txt', "Inference", 0, 100)    

#### Submit to an edge compute node with Intel® Xeon® Gold 6258R CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Gold 6258R Processor](https://ark.intel.com/content/www/us/en/ark/products/199350/intel-xeon-gold-6258r-processor-38-5m-cache-2-70-ghz.html). The inference workload will run on the CPU.

In [ ]:
job_id_xeon_cascade_lake = !qsub prediction.sh -l nodes=1:idc018 -F "results/xeon_cascade_lake/ CPU" -N arrhythmia_xeon_cascade_lake -e results/xeon_cascade_lake/ -o results/xeon_cascade_lake/
print(job_id_xeon_cascade_lake[0]) 
#Progress indicators
if job_id_xeon_cascade_lake:
    progressIndicator('./logs', job_id_xeon_cascade_lake[0] + '_load.txt', "Data Loading", 0, 100)
    progressIndicator('./logs', job_id_xeon_cascade_lake[0]+'.txt', "Inference", 0, 100)    

#### Submitting to an edge compute node with Intel® Xeon® E3-1268L v5 CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel 
    Xeon Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
job_id_xeon_skylake = !qsub prediction.sh -l nodes=1:idc007xv5 -F "results/xeon_skylake/ CPU" -N arrhythmia_xeon_skylake -e results/xeon_skylake/ -o results/xeon_skylake/
print(job_id_xeon_skylake[0]) 
#Progress indicators
if job_id_xeon_skylake:
    progressIndicator('./logs', job_id_xeon_skylake[0] + '_load.txt', "Data Loading", 0, 100)
    progressIndicator('./logs', job_id_xeon_skylake[0]+'.txt', "Inference", 0, 100)        

#### Submitting to an edge compute node with Intel® Core CPU and using the onboard Intel® GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
job_id_gpu = !qsub prediction.sh -l nodes=1:idc001skl -F "results/gpu/ GPU" -N arrhythmia_gpu -e results/gpu/ -o results/gpu/
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('./logs', job_id_gpu[0] + '_load.txt', "Data Loading", 0, 100)
    progressIndicator('./logs', job_id_gpu[0]+'.txt', "Inference", 0, 100)        

#### Submitting to an edge compute node with UP Squared Grove IoT Development Kit (UP2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/up-squared-grove-dev-kit">UP Squared Grove IoT Development Kit</a> edge node with an <a 
    href="https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz-">Intel® Atom® x7-E3950 Processor</a>. The inference  workload will run on the integrated Intel® HD Graphics 505 card.

In [ ]:
job_id_up2 = !qsub prediction.sh -l nodes=1:idc008u2g -F "results/up2/ GPU" -N arrhythmia_up2 -e results/up2/ -o results/up2/
print(job_id_up2[0]) 
#Progress indicators
if job_id_up2:
    progressIndicator('./logs', job_id_up2[0] + '_load.txt', "Data Loading", 0, 100)
    progressIndicator('./logs', job_id_up2[0]+'.txt', "Inference", 0, 100)    

### Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up. 

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job in step 2.3).
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 10 to 40 seconds each to complete.
</b></i></div> 

<br><div class=danger><b>Wait!: </b>Please wait for the inference jobs and video rendering to complete before proceeding to the next step to view results.</div>

## Compare Results

The running time of each inference task is recorded in `results/{arch}/stats_{job_id}.txt` and the output of the inference is stored at `results/{arch}/{job_name}.o{job_id}`. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('core2', 'Intel Core\ni7-8865UE\nCPU'),
             ('xeon_cascade_lake', 'Intel Xeon\nGold\n 6258R\nCPU'),
             ('xeon_skylake', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('up2', 'Intel Atom\nx7-E3950\nUP2/GPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/' + arch + '/stats_'+vars()['job_id_'+arch][0]+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

plt.ion()
summaryPlot(stats_list, 'Architecture', 'Time, miliseconds', 'Inference Engine Processing Time Per Sample', 'time' )

## Telemetry Dashboard
Once your submitted jobs are completed, run the cells below to view telemetry dashboards containing performance metrics for your model and target architecture.

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core™ i5-6500TE</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_core[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core™ i7-8865UE</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_core2[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Xeon® Gold 6258R CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_cascade_lake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Xeon® E3-1268L v5 CPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_xeon_skylake[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on Intel® Core CPU and using the onboard Intel® GPU</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_gpu[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

In [ ]:
link_t = "<a target='_blank' href='{href}'> Click here to view telemetry dashboard of the last job ran on UP Squared Grove IoT Development Kit (UP2)</a>"

result_file = "https://devcloud.intel.com/edge/metrics/d/" + job_id_up2[0].split('.')[0]

html = HTML(link_t.format(href=result_file))

display(html)

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge

## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>